# Importing the libraries

In [1]:
import numpy as np
import random
import os
import tensorflow as tf
from tensorflow import keras
import platform
from tensorflow.keras import layers, optimizers, losses, models, Input, Model
import time # Per misurare il tempo di training
from tensorflow.keras.callbacks import EarlyStopping # Per l'early stopping
import matplotlib.pyplot as plt
from tqdm import tqdm # Per mostrare una barra di progresso

# Set seeds for random operations.

In [2]:
# --- 1. Impostazione del Seed Globale all'inizio del tuo script ---
# Questo è il punto chiave per la riproducibilità di TUTTO ciò che segue.
MASTER_RANDOM_SEED = 42
np.random.seed(MASTER_RANDOM_SEED)
random.seed(MASTER_RANDOM_SEED) # Imposta anche il seed per la libreria 'random' di Python se la usi
tf.random.set_seed(MASTER_RANDOM_SEED)
os.environ['PYTHONHASHSEED'] = str(MASTER_RANDOM_SEED) # Per operazioni basate su hash (es. ordine dei dizionari)
os.environ['TF_DETERMINISTIC_OPS'] = '1' # Forza operazioni deterministiche in TensorFlow 2.x

# Print the HW Specs.

In [3]:
print("--- Dettagli dell'Architettura Hardware della Sessione Colab ---\n")

# --- 1. Dettagli CPU ---
print("--- Dettagli CPU ---")
!lscpu
print("\n")

# --- 2. Dettagli RAM (Memoria) ---
print("--- Dettagli RAM (Memoria) ---")
!cat /proc/meminfo | grep MemTotal
print("\n")

# --- 3. Dettagli Spazio su Disco ---
print("--- Dettagli Spazio su Disco ---")
!df -h /
print("\n")

# --- 4. Dettagli Acceleratore Hardware (GPU/TPU) ---
print("--- Dettagli Acceleratore Hardware (GPU/TPU) ---")
try:
    tpu_address = os.environ.get('COLAB_TPU_ADDR')
    if tpu_address:
        resolver = tf.distribute.cluster_resolver.TPUClusterResolver(tpu=tpu_address)
        tf.config.experimental_connect_to_cluster(resolver)
        tf.tpu.experimental.initialize_tpu_system(resolver)
        print(f"Tipo Acceleratore: TPU (indirizzo: {tpu_address})")
        print("Dispositivi TPU disponibili:")
        for device in tf.config.list_logical_devices('TPU'):
            print(f"  - {device.name}")
    else:
        gpus = tf.config.list_physical_devices('GPU')
        if gpus:
            print(f"Tipo Acceleratore: GPU")
            for gpu in gpus:
                print(f"  - Dispositivo GPU rilevato: {gpu.name}")
            print("\nDettagli GPU specifici (da `!nvidia-smi`):")
            !nvidia-smi
        else:
            print("Tipo Acceleratore: Nessuna GPU o TPU rilevata (in uso CPU)")

except Exception as e:
    print(f"Si è verificato un errore durante la rilevazione dell'acceleratore: {e}")
    print("Tentativo di rilevare i dispositivi TensorFlow standard:")
    devices = tf.config.list_logical_devices()
    if devices:
        for device in devices:
            print(f"  - Dispositivo rilevato: {device.name}, Tipo: {device.device_type}")
    else:
        print("Nessun dispositivo TensorFlow rilevato.")

print("\n--- Analisi Dettagli Hardware Completata ---")

--- Dettagli dell'Architettura Hardware della Sessione Colab ---

--- Dettagli CPU ---
Architecture:             x86_64
  CPU op-mode(s):         32-bit, 64-bit
  Address sizes:          46 bits physical, 48 bits virtual
  Byte Order:             Little Endian
CPU(s):                   2
  On-line CPU(s) list:    0,1
Vendor ID:                GenuineIntel
  Model name:             Intel(R) Xeon(R) CPU @ 2.00GHz
    CPU family:           6
    Model:                85
    Thread(s) per core:   2
    Core(s) per socket:   1
    Socket(s):            1
    Stepping:             3
    BogoMIPS:             4000.39
    Flags:                fpu vme de pse tsc msr pae mce cx8 apic sep mtrr pge m
                          ca cmov pat pse36 clflush mmx fxsr sse sse2 ss ht sysc
                          all nx pdpe1gb rdtscp lm constant_tsc rep_good nopl xt
                          opology nonstop_tsc cpuid tsc_known_freq pni pclmulqdq
                           ssse3 fma cx16 pcid sse4_1 sse4

# Connect To Gdrive to store the datasets created.

In [4]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


# Define the paths

In [7]:
# Percorsi dataset
paths = {
    "0-20": {
        "train": "/content/drive/MyDrive/GitHub/AWGN/dataset/UNIFORM_SNR_SAMPLES_training_0-20_SNR_50_BETA.npz",
        "val": "/content/drive/MyDrive/GitHub/AWGN/dataset/UNIFORM_SNR_SAMPLES_validation_0-20_SNR_50_BETA.npz",
    },
    "11-15": {
        "train": "/content/drive/MyDrive/GitHub/AWGN/dataset/training_11-15_SNR_50_BETA.npz",
        "val": "/content/drive/MyDrive/GitHub/AWGN/dataset/validation_11-15_SNR_50_BETA.npz",
    }
}

# Directory salvataggio modelli
save_dir = "/content/drive/MyDrive/GitHub/AWGN/trained_models/benchmarks"
os.makedirs(save_dir, exist_ok=True)

# Load the datasets.

In [6]:
def load_dataset(filepath):
    """
    Carica i dati X e y da un file .npz.
    Si aspetta che il file contenga 'X_train' o 'X_val' e 'y_train' o 'y_val'.
    """
    data = np.load(filepath)

    # Controlla se le chiavi per il training sono presenti
    if 'X_train' in data and 'y_train' in data:
        print(f"  Caricato Training data da: {filepath}")
        return data['X_train'], data['y_train']
    # Altrimenti, controlla se le chiavi per la validation sono presenti
    elif 'X_val' in data and 'y_val' in data:
        print(f"  Caricato Validation data da: {filepath}")
        return data['X_val'], data['y_val']
    else:
        # Se nessuna delle combinazioni attese è trovata, solleva un errore
        raise ValueError(f"Il file {filepath} non contiene i dati X e y attesi (né 'X_train'/'y_train' né 'X_val'/'y_val'). "
                         f"Chiavi trovate: {list(data.keys())}")

# Benchmark models definition

In [30]:
# Funzioni modello

def get_tdnn_model(input_shape, model_name='TDNN OFDM-DCSK'):
    # Dal paper: "Reliable and Secure Deep Learning-Based OFDM-DCSK Transceiver Design Without Delivery of Reference Chaotic Sequences"
    inputs = Input(shape=input_shape)
    input_length = input_shape[0]
    x = layers.Reshape((input_length, 1), name="reshape_for_conv")(inputs)
    x = layers.Conv1D(4, 2, activation='relu', padding='same', name="tdnn_conv1")(x)
    x = layers.Conv1D(8, 4, activation='relu', padding='same', name="tdnn_conv2")(x)
    x = layers.Conv1D(16, 8, activation='relu', padding='same', name="tdnn_conv3")(x)
    x = layers.Bidirectional(layers.LSTM(64, return_sequences=False), name="bidirectional_lstm")(x)
    x = layers.Dense(96, name="fc1")(x)
    x = layers.BatchNormalization(name="batch_norm")(x)
    x = layers.Activation('relu', name="relu_fc1")(x)
    outputs = layers.Dense(2, activation='softmax', name="output_softmax")(x)
    model = Model(inputs, outputs, name='DNN_Aided_Demodulator')
    model.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])
    return model

def get_mc_dlcs_model(input_shape, model_name='MC-DLCSK'):
    # Dal paper: "A Multi-Carrier Deep Learning CSK Communication System"
    if input_shape[0] % 2 != 0:
        raise ValueError("input_shape deve essere pari per MC-DLCSK")
    inputs = Input(shape=input_shape)
    x = layers.Reshape((input_shape[0], 1), name="reshape_for_lstm")(inputs)
    x = layers.Bidirectional(layers.LSTM(10, return_sequences=False), name="bidirectional_lstm")(x)
    x = layers.Dense(2, activation='relu', name="fc1_relu")(x)
    outputs = layers.Dense(2, activation='softmax', name="output_softmax")(x)
    model = Model(inputs, outputs, name=model_name)
    model.compile(optimizer=tf.keras.optimizers.SGD(0.01), loss='sparse_categorical_crossentropy', metrics=['accuracy'])
    return model

def get_lstm_dnn_model(input_shape, model_name='LSTM_DNN_OFDM_DCSK'):
    # Dal paper: "Intelligent and Reliable Deep Learning LSTM Neural Networks-Based OFDM-DCSK Demodulation Design"
    inputs = Input(shape=input_shape)
    if len(input_shape) == 1:
        x = layers.Reshape((1, input_shape[0]))(inputs)
    elif len(input_shape) == 2:
        x = inputs
    else:
        raise ValueError("input_shape deve essere 1D o 2D")
    # For K=32 (max length in the paper) authors set 77 LSTM units. In Table 1 69 units for the first Dense Layer.
    x = layers.LSTM(19)(x)
    x = layers.Dense(69, activation='relu')(x)
    x = layers.BatchNormalization()(x)
    outputs = layers.Dense(2, activation='softmax')(x)
    model = Model(inputs, outputs, name=model_name)
    model.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])
    return model

models_dict = {
    "TDNN OFDM-DCSK": get_tdnn_model,
    "MC-DLCSK": get_mc_dlcs_model,
    "LSTM_DNN_OFDM_DCSK": get_lstm_dnn_model
}

# Training...

In [ ]:
# Addestramento
for snr_range, datasets in paths.items():
    x_train, y_train = load_dataset(datasets['train'])
    x_val, y_val = load_dataset(datasets['val'])

    for model_name, builder in models_dict.items():
        print(f"\nAddestramento modello: {model_name} | SNR range: {snr_range}")
        model = builder(input_shape=x_train.shape[1:])

        callback = EarlyStopping(monitor='val_loss', patience=20, restore_best_weights=True)

        start_time = time.time() # Registra il tempo di inizio training

        # According to the papers.
        if model_name=="MC-DLCSK":
          epochs_nr = 20
        else:
          epochs_nr = 30

        if model_name=="TDNN OFDM-DCSK":
          batch = 50
        elif model_name=="LSTM_DNN_OFDM_DCSK":
          batch = 20


        model.fit(
            x_train, y_train,
            validation_data=(x_val, y_val),
            epochs=epochs_nr,
            batch_size=batch,
            callbacks=[callback],
            verbose=1
        )

        end_time = time.time() # Registra il tempo di fine training
        total_training_time = end_time - start_time

        print(f"Training completato.")
        print(f"Tempo totale di training: {total_training_time:.2f} secondi.") # Stampa il tempo totale

        save_path = os.path.join(save_dir, f"{model_name.replace(' ', '_')}-ORIGINAL-50-BETA_awgn_snr_{snr_range}.h5")
        model.save(save_path)
        print(f"Salvato modello in: {save_path}")

  Caricato Training data da: /content/drive/MyDrive/GitHub/AWGN/dataset/UNIFORM_SNR_SAMPLES_training_0-20_SNR_50_BETA.npz
  Caricato Validation data da: /content/drive/MyDrive/GitHub/AWGN/dataset/UNIFORM_SNR_SAMPLES_validation_0-20_SNR_50_BETA.npz

Addestramento modello: TDNN OFDM-DCSK | SNR range: 0-20
Epoch 1/30
2100/2100 ━━━━━━━━━━━━━━━━━━━━ 41s 18ms/step - accuracy: 0.9948 - loss: 0.0152 - val_accuracy: 1.0000 - val_loss: 2.6936e-06
Epoch 2/30
2100/2100 ━━━━━━━━━━━━━━━━━━━━ 51s 22ms/step - accuracy: 1.0000 - loss: 8.3387e-06 - val_accuracy: 1.0000 - val_loss: 3.5915e-07
Epoch 3/30
2100/2100 ━━━━━━━━━━━━━━━━━━━━ 37s 18ms/step - accuracy: 1.0000 - loss: 1.8229e-06 - val_accuracy: 1.0000 - val_loss: 8.5842e-08
Epoch 4/30
2100/2100 ━━━━━━━━━━━━━━━━━━━━ 40s 17ms/step - accuracy: 1.0000 - loss: 5.1968e-07 - val_accuracy: 1.0000 - val_loss: 3.2924e-10
Epoch 5/30
2100/2100 ━━━━━━━━━━━━━━━━━━━━ 51s 22ms/step - accuracy: 0.9998 - loss: 8.2309e-04 - val_accuracy: 1.0000 - val_loss: 1.1483e-06

Training completato.
Tempo totale di training: 1674.05 secondi.
Salvato modello in: /content/drive/MyDrive/GitHub/AWGN/trained_models/benchmarks/TDNN_OFDM-DCSK-ORIGINAL-50-BETA_awgn_snr_0-20.h5

Addestramento modello: MC-DLCSK | SNR range: 0-20
Epoch 1/20
2100/2100 ━━━━━━━━━━━━━━━━━━━━ 31s 14ms/step - accuracy: 0.6503 - loss: 0.6370 - val_accuracy: 0.9550 - val_loss: 0.2080
Epoch 2/20
2100/2100 ━━━━━━━━━━━━━━━━━━━━ 28s 13ms/step - accuracy: 0.9448 - loss: 0.1957 - val_accuracy: 0.7110 - val_loss: 0.5476
Epoch 3/20
2100/2100 ━━━━━━━━━━━━━━━━━━━━ 41s 13ms/step - accuracy: 0.7525 - loss: 0.4818 - val_accuracy: 0.9693 - val_loss: 0.1252
Epoch 4/20
2100/2100 ━━━━━━━━━━━━━━━━━━━━ 41s 13ms/step - accuracy: 0.9417 - loss: 0.1867 - val_accuracy: 0.9813 - val_loss: 0.0696
Epoch 5/20
2100/2100 ━━━━━━━━━━━━━━━━━━━━ 41s 13ms/step - accuracy: 0.9699 - loss: 0.0982 - val_accuracy: 0.9974 - val_loss: 0.0115
Epoch 6/20
2100/2100 ━━━━━━━━━━━━━━━━━━━━ 41s 13ms/step - accuracy: 0.9873 - loss: 0.0500 - val

Training completato.
Tempo totale di training: 756.14 secondi.
Salvato modello in: /content/drive/MyDrive/GitHub/AWGN/trained_models/benchmarks/MC-DLCSK-ORIGINAL-50-BETA_awgn_snr_0-20.h5

Addestramento modello: LSTM_DNN_OFDM_DCSK | SNR range: 0-20
Epoch 1/30
5250/5250 ━━━━━━━━━━━━━━━━━━━━ 42s 8ms/step - accuracy: 0.9915 - loss: 0.0275 - val_accuracy: 1.0000 - val_loss: 7.1199e-06
Epoch 2/30
5250/5250 ━━━━━━━━━━━━━━━━━━━━ 46s 9ms/step - accuracy: 0.9999 - loss: 4.5159e-04 - val_accuracy: 1.0000 - val_loss: 1.5750e-05
Epoch 3/30
5250/5250 ━━━━━━━━━━━━━━━━━━━━ 82s 9ms/step - accuracy: 0.9999 - loss: 3.1759e-04 - val_accuracy: 1.0000 - val_loss: 4.9100e-06
Epoch 4/30
5250/5250 ━━━━━━━━━━━━━━━━━━━━ 75s 8ms/step - accuracy: 0.9999 - loss: 2.3802e-04 - val_accuracy: 0.9999 - val_loss: 2.1288e-04
Epoch 5/30
5250/5250 ━━━━━━━━━━━━━━━━━━━━ 39s 7ms/step - accuracy: 0.9999 - loss: 4.9123e-04 - val_accuracy: 1.0000 - val_loss: 2.0618e-06
Epoch 6/30
5250/5250 ━━━━━━━━━━━━━━━━━━━━ 46s 9ms/step - accu

Training completato.
Tempo totale di training: 1674.25 secondi.
Salvato modello in: /content/drive/MyDrive/GitHub/AWGN/trained_models/benchmarks/LSTM_DNN_OFDM_DCSK-ORIGINAL-50-BETA_awgn_snr_0-20.h5
  Caricato Training data da: /content/drive/MyDrive/GitHub/AWGN/dataset/training_11-15_SNR_50_BETA.npz
  Caricato Validation data da: /content/drive/MyDrive/GitHub/AWGN/dataset/validation_11-15_SNR_50_BETA.npz

Addestramento modello: TDNN OFDM-DCSK | SNR range: 11-15
Epoch 1/30
1200/1200 ━━━━━━━━━━━━━━━━━━━━ 22s 15ms/step - accuracy: 0.9894 - loss: 0.0282 - val_accuracy: 1.0000 - val_loss: 3.1263e-06
Epoch 2/30
1200/1200 ━━━━━━━━━━━━━━━━━━━━ 17s 14ms/step - accuracy: 1.0000 - loss: 1.0830e-05 - val_accuracy: 1.0000 - val_loss: 5.9713e-07
Epoch 3/30
1200/1200 ━━━━━━━━━━━━━━━━━━━━ 21s 15ms/step - accuracy: 1.0000 - loss: 3.1830e-06 - val_accuracy: 1.0000 - val_loss: 1.7890e-07
Epoch 4/30
1200/1200 ━━━━━━━━━━━━━━━━━━━━ 17s 14ms/step - accuracy: 1.0000 - loss: 1.2676e-06 - val_accuracy: 1.0000 -

Training completato.
Tempo totale di training: 567.20 secondi.
Salvato modello in: /content/drive/MyDrive/GitHub/AWGN/trained_models/benchmarks/TDNN_OFDM-DCSK-ORIGINAL-50-BETA_awgn_snr_11-15.h5

Addestramento modello: MC-DLCSK | SNR range: 11-15
Epoch 1/20
1200/1200 ━━━━━━━━━━━━━━━━━━━━ 15s 12ms/step - accuracy: 0.4995 - loss: 0.6932 - val_accuracy: 0.4989 - val_loss: 0.6932
Epoch 2/20
1200/1200 ━━━━━━━━━━━━━━━━━━━━ 19s 10ms/step - accuracy: 0.5026 - loss: 0.6932 - val_accuracy: 0.4989 - val_loss: 0.6932
Epoch 3/20
1200/1200 ━━━━━━━━━━━━━━━━━━━━ 12s 10ms/step - accuracy: 0.5025 - loss: 0.6932 - val_accuracy: 0.4989 - val_loss: 0.6932
Epoch 4/20
1200/1200 ━━━━━━━━━━━━━━━━━━━━ 12s 10ms/step - accuracy: 0.5025 - loss: 0.6932 - val_accuracy: 0.4989 - val_loss: 0.6932
Epoch 5/20
1200/1200 ━━━━━━━━━━━━━━━━━━━━ 12s 10ms/step - accuracy: 0.5025 - loss: 0.6932 - val_accuracy: 0.4989 - val_loss: 0.6932
Epoch 6/20
1200/1200 ━━━━━━━━━━━━━━━━━━━━ 13s 11ms/step - accuracy: 0.5025 - loss: 0.6932 - va

Training completato.
Tempo totale di training: 291.52 secondi.
Salvato modello in: /content/drive/MyDrive/GitHub/AWGN/trained_models/benchmarks/MC-DLCSK-ORIGINAL-50-BETA_awgn_snr_11-15.h5

Addestramento modello: LSTM_DNN_OFDM_DCSK | SNR range: 11-15
Epoch 1/30
3000/3000 ━━━━━━━━━━━━━━━━━━━━ 20s 6ms/step - accuracy: 0.9847 - loss: 0.0495 - val_accuracy: 0.9999 - val_loss: 2.2719e-04
Epoch 2/30
3000/3000 ━━━━━━━━━━━━━━━━━━━━ 19s 6ms/step - accuracy: 0.9996 - loss: 0.0014 - val_accuracy: 0.9793 - val_loss: 0.0653
Epoch 3/30
3000/3000 ━━━━━━━━━━━━━━━━━━━━ 22s 6ms/step - accuracy: 0.9999 - loss: 7.0295e-04 - val_accuracy: 1.0000 - val_loss: 5.4557e-06
Epoch 4/30
3000/3000 ━━━━━━━━━━━━━━━━━━━━ 17s 6ms/step - accuracy: 0.9999 - loss: 2.9640e-04 - val_accuracy: 1.0000 - val_loss: 3.7444e-07
Epoch 5/30
3000/3000 ━━━━━━━━━━━━━━━━━━━━ 23s 7ms/step - accuracy: 1.0000 - loss: 2.2166e-05 - val_accuracy: 0.9999 - val_loss: 9.4779e-05
Epoch 6/30
3000/3000 ━━━━━━━━━━━━━━━━━━━━ 17s 6ms/step - accuracy: 